In [ ]:
import configparser
import webbrowser
import os
import requests
import base64
import PyPDF2
import re
import msal

In [ ]:
config = configparser.ConfigParser()
config.read(['config.cfg', 'config.dev.cfg'])
azure_settings = config['azure']

In [ ]:
client_id = azure_settings['clientId']
tenant_id = azure_settings['tenantId']
scopes = azure_settings['graphUserScopes'].split(' ')

In [ ]:
access_token_cache = msal.SerializableTokenCache()

# Getting access token for particular account

In [ ]:
client = msal.PublicClientApplication(client_id, token_cache=access_token_cache)
flow = client.initiate_device_flow(scopes=scopes)

In [ ]:
verification_uri = flow['verification_uri']
user_code = flow['user_code']

# webbrowser.open(verification_uri)
print(flow['message'])

In [ ]:
access_token = client.acquire_token_by_device_flow(flow)

with open('access_token.json', 'w') as f:
    f.write(access_token_cache.serialize())

In [ ]:
access_token

# Reading from saved json

In [ ]:
if os.path.exists('access_token.json'):
    with open('access_token.json', 'r') as f:
        access_token_cache.deserialize(f.read())

In [ ]:
client = msal.PublicClientApplication(client_id, token_cache=access_token_cache)
# flow = client.initiate_device_flow(scopes=scopes)

In [ ]:
accounts = client.get_accounts()[0]
accounts

In [ ]:
token = client.acquire_token_silent(scopes, account=accounts)
token

In [ ]:
headers = {
        'Authorization': 'Bearer ' + token['access_token']
    }

In [ ]:
Graph_API_Endpoint ='https://graph.microsoft.com/v1.0'
endpoint = Graph_API_Endpoint + '/me/photo/$value'

In [ ]:
response = requests.get(endpoint, headers=headers)

In [ ]:
response

In [ ]:
with open('photo.png', 'wb') as f:
    f.write(response.content)

## Reading the mail

In [ ]:
headers = {
        'Authorization': 'Bearer ' + token['access_token']
    }

In [ ]:
Graph_API_Endpoint ='https://graph.microsoft.com/v1.0'
endpoint = Graph_API_Endpoint + '/me/messages?$search="pizza bill"'

In [ ]:
response = requests.get(endpoint, headers=headers)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
mailId = response.json()['value'][0]['id']
mailId

### Downloading Attachment

In [ ]:
headers = {
        'Authorization': 'Bearer ' + token['access_token']
    }

In [ ]:
Graph_API_Endpoint ='https://graph.microsoft.com/v1.0'
endpoint = Graph_API_Endpoint + f'/me/messages/{mailId}/attachments'

In [ ]:
response = requests.get(endpoint, headers=headers)

In [ ]:
fileAttachments = response.json()['value'][0]

In [ ]:
"""
Another way of getting each attachment by there respective id
"""

# attchmentId = fileAttachments['id']
# Graph_API_Endpoint ='https://graph.microsoft.com/v1.0'
# # endpoint = Graph_API_Endpoint + f'/me/messages/{mailId}/attachments/{attchmentId}'
# # for just the value
# endpoint = Graph_API_Endpoint + f'/me/messages/{mailId}/attachments/{attchmentId}/$value'
# response = requests.get(endpoint, headers=headers)
# # response.json()
# # response.content

In [ ]:
with open('./response_downloads/' + fileAttachments['name'], 'wb') as f:
    f.write(base64.b64decode(fileAttachments['contentBytes']))
    # f.write(response.content)

### finding values inside pdf

In [ ]:
pdfFileName = "./response_downloads/Gmail - Domino's Pizza order confirmation.pdf"

In [ ]:
searchText = '480221567'

In [ ]:
pdfFile = PyPDF2.PdfFileReader(pdfFileName)

In [ ]:
totalPages = pdfFile.getNumPages()
totalPages

In [ ]:
for i in range(totalPages):
    page = pdfFile.getPage(i)
    pageContent = page.extractText()
    if searchText in pageContent:
        print(f"Found {searchText} on page {i+1}")
        # print(pageContent)